In [20]:
# 1. Importer les bibliothèques nécessaires
import pandas as pd
import json
import folium
from shapely.geometry import Polygon
import os

os.makedirs('../generated/sites', exist_ok=True)

In [33]:
# 2. Charger les données tabulaires et géospatiales
faits = pd.read_csv('../generated/faits_matieres.csv')
dim_type_matiere = pd.read_csv('../generated/dim_type_matiere.csv')
dim_producteur = pd.read_csv('../generated/dim_producteur.csv')
dim_temps = pd.read_csv('../generated/dim_temps.csv')
dim_site_valorisation = pd.read_csv('../generated/dim_site_valorisation.csv')

geojson_path = "../dataset/collecte.json"
with open(geojson_path, "r") as f:
    geo_data = json.load(f)

features = geo_data['features']
geo_df = pd.json_normalize(features)
# Ajouter la colonne PROVINCE='Quebec' si MUNICIPALITE == 'Montreal' dans geo_df
if 'properties.MUNICIPALITE' in geo_df.columns:
    geo_df.loc[geo_df['properties.MUNICIPALITE'] == 'Montreal', 'properties.PROVINCE'] = 'Québec'
print("Colonnes de geo_df avant sélection : \n")
print(geo_df.columns)
print("Valeurs uniques de PROVINCE dans geo_df :")

geo_df = geo_df[['properties.PROVINCE', 'properties.JOUR', 'geometry.type', 'geometry.coordinates']]
geo_df.head()

# 3. Fusionner les données sur le secteur
faits_geo = faits.merge(geo_df, left_on='geo', right_on='properties.PROVINCE', how='left')
# faits_geo.head(500)

# 4. Visualiser la carte interactive
m = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
for _, row in geo_df.iterrows():
    if row['geometry.type'] == 'Polygon':
        folium.Polygon(
            locations=[(lat, lon) for lon, lat in row['geometry.coordinates'][0]],
            color='blue',
            fill=True,
            fill_opacity=0.2,
            popup=row['properties.PROVINCE']
        ).add_to(m)
# Ajouter un marker pour chaque fait (si coordonnées disponibles)
for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        # print(row['geo'], row['type_matiere'], row['volume_tonnes'])

        # Extraire tous les points du polygone
        coords = row['geometry.coordinates'][0]
        # Créer le polygone et calculer le centroïde
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}",
            icon=folium.Icon(color='red', icon='home')
        ).add_to(m)
m.save("../generated/sites/collecte_secteurs.html")
print("Carte interactive générée avec markers : collecte_secteurs.html")


Colonnes de geo_df avant sélection : 

Index(['type', 'properties.ID', 'properties.MUNICIPALITE',
       'properties.SECTEUR', 'properties.JOUR', 'properties.FREQUENCE',
       'properties.TYPE_DECHET', 'properties.MESSAGE_FR',
       'properties.MESSAGE_EN', 'properties.EXCEPTION_FR',
       'properties.EXCEPTION_EN', 'geometry.type', 'geometry.coordinates',
       'properties.PROVINCE'],
      dtype='object')
Valeurs uniques de PROVINCE dans geo_df :
Carte interactive générée avec markers : collecte_secteurs.html
Carte interactive générée avec markers : collecte_secteurs.html


# Visualisations géospatiales croisées avancées
Ce qui suit présente plusieurs exemples de visualisations géospatiales croisées :
- Carte choroplèthe (volume par secteur)
- Heatmap de points (volume ou fréquence)
- Cluster de markers (densité)
- Animation temporelle (évolution par période)
- Flux de matières (producteur → site)
- Densité par type de matière
- Points chauds (hotspots)

In [ ]:
# Carte choroplèthe : volume total par secteur
import folium
import pandas as pd
import json
from branca.colormap import linear

# Agréger le volume par secteur
volume_par_secteur = faits.groupby('geo')['volume_tonnes'].sum().reset_index()

# Charger le geojson
with open(geojson_path, "r") as f:
    geo_data = json.load(f)

# Créer la carte
m_choro = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

# Créer le colormap
colormap = linear.Blues_09.scale(volume_par_secteur['volume_tonnes'].min(), volume_par_secteur['volume_tonnes'].max())
colormap.caption = 'Volume total par secteur'
folium.GeoJson(
    geo_data,
    style_function=lambda feature: {
        'fillColor': colormap(volume_par_secteur.set_index('geo').get(feature['properties']['PROVINCE'], 0)),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(fields=['PROVINCE'])
).add_to(m_choro)
colormap.add_to(m_choro)
m_choro.save('../generated/sites/choropleth_volume_secteur.html')
print('Carte choroplèthe générée : choropleth_volume_secteur.html')

Carte choroplèthe générée : choropleth_volume_secteur.html


In [ ]:
# Heatmap de points : volume ou fréquence de collecte
from folium.plugins import HeatMap
import numpy as np

# Préparer les points (centroïdes des polygones avec volume)
points = []
for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        coords = row['geometry.coordinates'][0]
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        pt = [centroid.y, centroid.x, row['volume_tonnes']]
        # Vérifier qu'il n'y a pas de NaN
        if not any(np.isnan(pt)):
            points.append(pt)

# Créer la carte heatmap
m_heat = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
HeatMap(points, radius=15, blur=10, min_opacity=0.4, max_zoom=1).add_to(m_heat)
m_heat.save('../generated/sites/heatmap_volume.html')
print('Heatmap générée : heatmap_volume.html')

Heatmap générée : heatmap_volume.html


In [ ]:
# Cluster de markers : densité de faits
from folium.plugins import MarkerCluster

m_cluster = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m_cluster)

for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        coords = row['geometry.coordinates'][0]
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}",
            icon=folium.Icon(color='green', icon='info-sign')
        ).add_to(marker_cluster)

m_cluster.save('../generated/sites/cluster_markers.html')
print('Cluster de markers généré : cluster_markers.html')

Cluster de markers généré : cluster_markers.html


In [ ]:
# Animation temporelle : évolution du volume par période
from folium.plugins import TimestampedGeoJson

# Préparer les features pour chaque période
features = []
for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        coords = row['geometry.coordinates'][0]
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [centroid.x, centroid.y]
            },
            'properties': {
                'time': row['periode'],
                'popup': f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}"
            }
        }
        features.append(feature)

geojson_dict = {
    'type': 'FeatureCollection',
    'features': features
}

m_time = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
TimestampedGeoJson(
    geojson_dict,
    period='P1Y',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True
).add_to(m_time)
m_time.save('../generated/sites/animation_temporelle.html')
print('Animation temporelle générée : animation_temporelle.html')

Animation temporelle générée : animation_temporelle.html


In [ ]:
# Flux de matières : lignes entre producteurs et sites de valorisation
m_flux = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

for _, row in faits.iterrows():
    # Exemple : coordonnées fictives pour producteurs et sites (à remplacer par les vraies coordonnées si disponibles)
    prod_coords = [row.get('lat_producteur', 45.5), row.get('lon_producteur', -73.56)]
    site_coords = [row.get('lat_site', 45.52), row.get('lon_site', -73.58)]
    folium.Marker(location=prod_coords, popup=f"Producteur: {row.get('producteur', 'Inconnu')}", icon=folium.Icon(color='blue')).add_to(m_flux)
    folium.Marker(location=site_coords, popup=f"Site: {row.get('site_valorisation', 'Inconnu')}", icon=folium.Icon(color='orange')).add_to(m_flux)
    folium.PolyLine([prod_coords, site_coords], color='purple', weight=3, opacity=0.7).add_to(m_flux)

m_flux.save('../generated/sites/flux_matieres.html')
print('Carte des flux de matières générée : flux_matieres.html')

Carte des flux de matières générée : flux_matieres.html


In [ ]:
# Densité par type de matière : markers colorés par type
m_density = folium.Map(location=[45.5017, -73.5673], zoom_start=11)

type_colors = {
    'Plastique': 'red',
    'Métal': 'blue',
    'Papier': 'green',
    'Verre': 'purple',
    'Organique': 'orange',
    # Ajouter d'autres types si nécessaire
}

for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        coords = row['geometry.coordinates'][0]
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        color = type_colors.get(row['type_matiere'], 'gray')
        folium.Marker(
            location=[centroid.y, centroid.x],
            popup=f"{row['type_matiere']} ({row['volume_tonnes']} t) - {row['geo']}",
            icon=folium.Icon(color=color, icon='info-sign')
        ).add_to(m_density)

m_density.save('../generated/sites/densite_type_matiere.html')
print('Carte densité par type de matière générée : densite_type_matiere.html')

Carte densité par type de matière générée : densite_type_matiere.html


In [ ]:
# Points chauds (hotspots) par volume
import numpy as np

# Préparer les points avec volume
hot_points = []
for _, row in faits_geo.iterrows():
    if row['geometry.type'] == 'Polygon' and isinstance(row['geometry.coordinates'], list):
        coords = row['geometry.coordinates'][0]
        poly = Polygon([(lon, lat) for lon, lat in coords])
        centroid = poly.centroid
        hot_points.append([centroid.y, centroid.x, row['volume_tonnes']])

# Définir un seuil pour les hotspots (exemple : top 10 %)
volumes = np.array([p[2] for p in hot_points])
seuil = np.percentile(volumes, 90)
hotspots = [p for p in hot_points if p[2] >= seuil]

m_hotspot = folium.Map(location=[45.5017, -73.5673], zoom_start=11)
for pt in hotspots:
    folium.CircleMarker(
        location=[pt[0], pt[1]],
        radius=10,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=f'Hotspot: {pt[2]} t'
    ).add_to(m_hotspot)

m_hotspot.save('../generated/sites/hotspots_volume.html')
print('Carte des hotspots générée : hotspots_volume.html')

Carte des hotspots générée : hotspots_volume.html
